#### Load data

In [1]:
with open('log/IIIHTTP2.log','r') as InputData:
    APPdata = InputData.readlines()
    APPdata = [i.strip().split('\t') for i in APPdata]
    APPdata = list(filter(lambda x:'#' not in x[0],APPdata))

In [2]:
with open('log/IIIHTTP3.log','r') as InputData2:
    Testdata = InputData2.readlines()
    Testdata = [i.strip().split('\t') for i in Testdata]
    Testdata = list(filter(lambda x:'#' not in x[0],Testdata))

In [3]:
def similarity(test_data,whiteList_data):
    score = 0
    for key in fingerprint_keys_index:
        for key_index in fingerprint_keys_index[key]:
            if test_data[key_index] in whiteList_data[key]:
                score += 1
    #if score > 1:
    #    WhiteListUpdate(test_data,whiteList_data)
    #else:
    #    outgoingInfo()
    return score

In [4]:
def WhiteListUpdate(data):
    pass

In [5]:
def outgoingInfo(label,data):
    req_len = data[13]
    res_len = data[14]
    #label = data[12]
    if req_len > trainedList[label]['request_len']:
        print(req_len,trainedList[label]['request_len'])
        #return 
    elif res_len > trainedList[label]['respond_len']:
        print(res_len,trainedList[label]['respond_len'])
    else:
        return 

In [6]:
def average_data_len(label,req_data,res_data,num):
    average_req = 0
    average_res = 0
    if req_data != 0:
        num[0] += 1
        average_req = (float(trainedList[label]['request_len']) *(num[0] - 1) + req_data)/num[0]
    if res_data != 0:
        num[1] += 1
        average_res = (float(trainedList[label]['respond_len']) *(num[1] - 1) + req_data)/num[1]
    return average_req,average_res,num

#### Training the white list fingerprint

In [7]:
fingerprint_keys_index={'ip':[4],'host':[8],'userAgent':[12]}
trainedList = {}
for index in range(len(APPdata)):
    if APPdata[index][12].strip() != "-":
        label = APPdata[index][12].replace('\'','').strip()
        if label not in trainedList:
            tmp_dict = {'ip':[APPdata[index][4].replace('\'','').strip()],
                        'host':[APPdata[index][8].replace('\'','').strip()],
                        'userAgent':[label],
                       'num':[0,0]}
            trainedList.setdefault(label)
            trainedList[label] = tmp_dict
            if int(APPdata[index][13].replace('\'','').strip()) != 0:
                trainedList[label]['request_len'] = float(APPdata[index][13].replace('\'','').strip())
            else:
                trainedList[label]['request_len'] = []
            if int(APPdata[index][14].replace('\'','').strip()) != 0:
                trainedList[label]['respond_len'] = float(APPdata[index][14].replace('\'','').strip())
            else:
                trainedList[label]['respond_len'] = []
        else:
            for key in fingerprint_keys_index:
                for key_index in fingerprint_keys_index[key]:
                    tmp_APPdata = APPdata[index][key_index].replace('\'','').strip()
                    if  tmp_APPdata not in trainedList[label][key]:
                        trainedList[label].setdefault(key,[]).append(tmp_APPdata) 
        trainedList[label]['request_len'],trainedList[label]['respond_len'],trainedList[label]['num'] = average_data_len(label,float(APPdata[index][13]),float(APPdata[index][14]),trainedList[label]['num'])

In [8]:
print(trainedList['Java/1.8.0_151'])

{'ip': ['210.241.131.253', '211.75.100.229'], 'host': ['tisvcloud.freeway.gov.tw', 'ptx.transportdata.tw'], 'userAgent': ['Java/1.8.0_151'], 'num': [0, 0], 'request_len': 0, 'respond_len': 0}


In [9]:
toBeDetermined = []
NoHost =[]
for test_index in range(len(Testdata)):
    similarityScore = 0
    if Testdata[test_index][8] == '-':
        NoHost.append(Testdata[test_index])
    else:
        if Testdata[test_index][12] in trainedList:
            whiteList = trainedList[Testdata[test_index][12]]
            similarityScore = similarity(Testdata[test_index],whiteList)
        else:
            max_score = 0
            likelihoodAPP = []
            for label in trainedList:
                similarityScore = similarity(Testdata[test_index],trainedList[label])
                if similarityScore > max_score:
                    likelihoodAPP = Testdata[test_index]
                    max_score = similarityScore
                elif (similarityScore > 0) and (similarityScore < max_score):
                    likelihoodAPP.append(Testdata[test_index])
            if max_score == 0:
                toBeDetermined.append(Testdata[test_index])
            #print(similarityScore,Testdata[test_index][12],likelihoodAPP)